In [ ]:
!pip install nevergrad
!pip install yellowbrick

import sqlite3 as db
import pandas as pd
import json
import time
import statistics as stat
import random

import matplotlib.pyplot as plt
import seaborn as sns
import nevergrad as ng

from copy import deepcopy
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, r2_score


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import BaggingRegressor

import yellowbrick
from yellowbrick.regressor import prediction_error, PredictionError

<center><h2><b>Leer DB</b></h2></center>

Primero de todo leemos la base de datos en un dataframe. Hemos limpiado cada tabla individualmente y las hemos juntado con LEFT JOIN para que no se pierda<br>
ninguna información de la tabla principal que es pacient.

In [ ]:
def sql_query(q):
    conn = db.connect('../db/sqlite/eicu_v2_0_1_clean.sqlite3')
    df = pd.read_sql_query(q, conn)
    conn.close()
    
    return df

In [ ]:
X_query = """
    SELECT *
    FROM patient P LEFT JOIN diagnosis                    D ON P.patientunitstayid = D.patientunitstayid
                   LEFT JOIN admissiondrug               AD ON P.patientunitstayid = AD.patientunitstayid
                   LEFT JOIN respiratoryCare             RC ON P.patientunitstayid = RC.patientunitstayid
                   LEFT JOIN physicalExam                PE ON P.patientunitstayid = PE.patientunitstayid
                   LEFT JOIN admissionDx                ADX ON P.patientunitstayid = ADX.patientunitstayid
                   LEFT JOIN carePlanCareProvider         C ON P.patientunitstayid = C.patientunitstayid
                   LEFT JOIN carePlanGeneral             CG ON P.patientunitstayid = CG.patientunitstayid
                   LEFT JOIN carePlanInfectiousDisease CGID ON P.patientunitstayid = CGID.patientunitstayid
                   LEFT JOIN carePlanGoal               CPG ON P.patientunitstayid = CPG.patientunitstayid
                   LEFT JOIN vitalAperiodic             VAP ON P.patientunitstayid = VAP.patientunitstayid
                   LEFT JOIN vitalPeriodic               VP ON P.patientunitstayid = VP.patientunitstayid
                   LEFT JOIN medication                   M ON P.patientunitstayid = M.patientunitstayid
                   LEFT JOIN allergy                     AL ON P.patientunitstayid = AL.patientunitstayid
                   LEFT JOIN infusiondrug                ID ON P.patientunitstayid = ID.patientunitstayid
"""

X = sql_query(X_query).drop(columns=['patientunitstayid'])
y = X['unitdischargeoffset']

X['apacheadmissiondx'] = X['apacheadmissiondx'].fillna('_Unknown')

<center><h2><b>Sanear valores dejados a NaN del Left Join</b></h2></center>

Al hacer Left Join nos quedan campos nulos. Dependiendo del tipo de datos y de los otros datos de cada columna en particular<br>
los rellenamos con 0, -1 o un string indicando falta de valor.

In [ ]:
# admissiondrug
X['currenthistoryseqnum'] = X['currenthistoryseqnum'].fillna('_Unknown')
for col in ['ASPIRIN', 'LISINOPRIL', 'LASIX']:
    X[col] = X[col].fillna(0.0)

# physicalExam
for col in ['BPD_Current', 'BPD_Highest', 'BPD_Lowest', 'BPS_Current', 'BPS_Highest', 'BPS_Lowest', 'Blood_Loss', 'Dialysis_Net', 'O2Sat_Current',
            'O2Sat_Highest', 'O2Sat_Lowest', 'Urine', 'Intubated', 'Comatose', 'Ventilated', 'Motor', 'Verbal', 'Eyes']:
    X[col] = X[col].fillna(-1)

# admissionDx
for col in ['Cardiovascular', 'Respiratory', 'Neurologic']:
    X[col] = X[col].fillna(0)

# carePlanGoal
for col in ['Care_Patient_Family', 'Care_Pulmonary', 'Care_Fluid_Balance_Treatments', 'Care_Activity_Safety',
            'Care_Cardiovascular', 'Care_Infection_Labs']:
    X[col] = X[col].fillna(0)

# carePlanCareProvider
for col in ['Categoria_Hospital', 'Categoria_Cardiology', 'Categoria_Internal_Medicine', 'Intervencion_I', 'Intervencion_II', 'Intervencion_III', 'Intervencion_IV']:
    X[col] = X[col].fillna(0)

# carePlanGeneral
for col in ['Categoria_Ventilacion', 'Categoria_DVT_Prophylaxis', 'Categoria_Airway', 'Categoria_Care_Limitation', 'Categoria_Stress_Ulcer_Prophylaxis']:
    X[col] = X[col].fillna(0)

# carePlanInfectiousDisease
for col in ['infectdiseasesite', 'infectdiseaseassessment']:
    X[col] = X[col].fillna('_None')

# vitalAperiodic
X['last_aperiodic_off'] = X['last_aperiodic_off'].fillna(-1)
for col in ['last_aperiodic_systolic', 'last_aperiodic_diastolic', 'last_aperiodic_mean']:
    X[col] = X[col].fillna(X[col].median())
    
# vitalPeriodic
for col in ['temperature', 'sao2', 'respiration', 'cvp', 'heartrate']:
    X[col] = X[col].fillna(X[col].median())

# Medication
X['last_order_offset'] = X['last_order_offset'].fillna(-1)

# Allergy
for col in ['nDrugsAllergic', 'nNondrugsAllergic', 'totalAllergic']:
    X[col] = X[col].fillna(0)
    X[col] = X[col].astype('int32')
    
# Diagnosis
for col in ['last1', 'last2', 'last3', 'last4']:
    X[col] = X[col].fillna('_Unknown')
for col in ['last1_off', 'last2_off', 'last3_off', 'last4_off']:
    X[col] = X[col].fillna(-1)

# InfusionDrug
X['lastInfusionDrugOffset'] = X['lastInfusionDrugOffset'].fillna(-1)

Tras el paso anterior, este es el dataset resultante:

In [ ]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 400)
X.head(10)

<center><h2><b>Transformación de columnas</b></h2></center>

In [ ]:
categs = {}

for col in ['gender', 'ethnicity', 'hospitalid', 'hospitaladmitsource', 'currenthistoryseqnum', 'infectdiseasesite', 'infectdiseaseassessment',
            'nDrugsAllergic', 'nNondrugsAllergic', 'totalAllergic']:
    categs[col] = list(set(X[col]))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=234750)

In [ ]:
# Condicionar el test según la edad del paciente

#X_test = X_test[X_test['age'] >= 60]
#y_test = y_test[y_test.index.isin(X_test.index)]

No hemos puesto demasiada antención a la eliminación de outliers y normalización de columnas numéricas ya que nuestro principal regresor ha sido RandomForest. Hemos usado<br>
OneHot para las variables categóricas con un número limitado de columnas y directamente passthrough para las variables numéricas.

In [ ]:
transformers = [
    # Patient
    ('gender',                  OneHotEncoder(categories=[categs['gender']]),  []),
    ('age',                     'passthrough',    []),
    ('ethnicity',               OneHotEncoder(categories=[categs['ethnicity']]),  []),
    ('hospitalid',              OneHotEncoder(categories=[categs['hospitalid']]),  []),
    ('apacheadmissiondx',       'drop',           []),
    ('admissionheight',         'passthrough',    []),
    ('hospitaladmitoffset',     'passthrough',    []),
    ('hospitaladmitsource',      OneHotEncoder(categories=[categs['hospitaladmitsource']]), []),
    ('hospitaldischargeoffset', 'drop',           []),
    ('unitvisitnumber',         'passthrough',    []),    
    ('admissionweight',         'passthrough',    []),
    ('unitdischargeoffset',     'drop',           []),
    ('_avg_unit_stay',          'passthrough',    []),
    ('_avg_hospital_stay',      'passthrough',    []),
    ('_admission_bmi',          'passthrough',    []),
    
    # Diagnosis
    ('_DIAGNOSIS_last1', 'drop', []),
    ('_DIAGNOSIS_last2', 'drop', []),
    ('_DIAGNOSIS_last3', 'drop', []),
    ('_DIAGNOSIS_last4', 'drop', []),


    ('_DIAGNOSIS_last1_off', 'passthrough', []),
    ('_DIAGNOSIS_last2_off', 'passthrough', []),
    ('_DIAGNOSIS_last3_off', 'passthrough', []),
    ('_DIAGNOSIS_last4_off', 'passthrough', []),
    
    # AdmissionDrug
    ('_admissionAspirin',    'passthrough', []),
    ('_admissionLisinopril', 'passthrough', []),
    ('_admissionLasix',      'passthrough', []),
    
    # RespCare
    ('currenthistoryseqnum', OneHotEncoder(categories=[categs['currenthistoryseqnum']]), []),
    
    # PhysicalExam
    # TODO
    ('a0', 'drop', []),
    ('a1', 'drop', []),
    ('a2', 'drop', []),
    ('a3', 'drop', []),
    ('a4', 'drop', []),
    ('a5', 'drop', []),
    ('a6', 'drop', []),
    ('a7', 'drop', []),
    ('a8', 'drop', []),
    ('a9', 'drop', []),
    ('b1', 'drop', []),    
    ('b2', 'drop', []),
    ('b3', 'drop', []),
    ('b4', 'drop', []),
    ('b5', 'drop', []),
    ('b6', 'drop', []),
    ('b7', 'drop', []),
    ('b8', 'drop', []),

    # AdmissionDx
    ('_admissionCardiovascular', 'passthrough',  []),
    ('_admissionRespiratory',    'passthrough',  []),
    ('_admissionNeurologic',     'passthrough',  []),
  
    # CarePlanProvider
    ('Categoria_Hospital',          'passthrough', []), 
    ('Categoria_Cardiology',        'passthrough', []),
    ('Categoria_Internal_Medicine', 'passthrough', []),
    ('Intervencion_I',              'passthrough', []),
    ('Intervencion_II',             'passthrough', []),
    ('Intervencion_III',            'passthrough', []), 
    ('Intervencion_IV',             'passthrough', []),
    
    # CarePlanGeneral
    ('Categoria_Ventilacion',              'passthrough', []),
    ('Categoria_DVT_Prophylaxis',          'passthrough', []),
    ('Categoria_Airway',                   'passthrough', []),
    ('Categoria_Care_Limitation',          'passthrough', []),
    ('Categoria_Stress_Ulcer_Prophylaxis', 'passthrough', []),
    
    # CarePlanInfectiousDisease
    ('infectdiseasesite',      OneHotEncoder(categories=[categs['infectdiseasesite']]), []),
    ('infectdiseaseassessment', OneHotEncoder(categories=[categs['infectdiseaseassessment']]), []),
    
    ('last_aperiodic_off',       'passthrough', []),
    ('last_aperiodic_systolic',  'passthrough', []),
    ('last_aperiodic_diastolic', 'passthrough', []),
    ('last_aperiodic_mean',      'passthrough', []),
    
    # vitalPeriodic
    ('VP_temp',        'passthrough', []),
    ('VP_sao2',        'passthrough', []),
    ('VP_respiration', 'passthrough', []),
    ('VP_cvp',         'passthrough', []),
    ('VP_heartrate',   'passthrough', []),

    # Medication
    ('last_order_offset', 'passthrough', []),
    
    # Allergy
    # TODO
    ('nDrugsAllergic',    'drop', []),#OneHotEncoder(categories=[categs['nDrugsAllergic']]), []),
    ('nNondrugsAllergic', 'drop', []),#OneHotEncoder(categories=[categs['nNondrugsAllergic']]), []),
    ('totalAllergic',     'drop', []),#OneHotEncoder(categories=[categs['totalAllergic']]), []),
    
    # TODO
    ('asdofibasodfboasdfdsf',  'passthrough', []),
    
]

# Numerar columnas para el tranformador
for i in range(len(transformers)):
    transformers[i][2].append(i)

# Transformar la matriz
X_T_train = ColumnTransformer(transformers=transformers).fit_transform(X_train)
X_T_test  = ColumnTransformer(transformers=transformers).fit_transform(X_test)

# Mostrar el cambio en columnas
print('Train', X.shape, '->', X_T_train.shape)
print('Test ', X_test.shape, ' ->', X_T_test.shape)

<center><h2><b>Busqueda de Hiperparametros</b></h2></center>

Hemos usado Nevergrad para buscar los hiperparámetros de RandomForest y XBoost mediante un algorítmo genético, ya que son una cantidad importante de hiperparámetros.<br>

In [ ]:
# Número aleatorio para la búsqueda de hiperparámetros
rand_n = random.randint(0, 100000000000)

# Funcion auxiliar para devolver un conjunto de scores y su media y desviación estandar
def cv_avg_std(reg, X, y, scoring):
    maes = cross_val_score(reg, X, y, cv=5, scoring=scoring)
    avg = stat.mean(maes)
    std_dev = stat.variance(maes)**(1/2)
    
    return maes, avg, std_dev

# Función de optimización para nevergrad
def optimize_spectral(n_estimators, min_samples_leaf, min_samples_split, max_depth, max_features, warm_start, bootstrap):
    reg = RandomForestRegressor(n_estimators=n_estimators, min_samples_leaf=min_samples_leaf,
                              min_samples_split=min_samples_split, max_depth=max_depth, random_state=rand_n,
                              max_features=max_features, bootstrap=bootstrap)

    maes, avg, std_dev = cv_avg_std(reg, X_T_train, y_train, 'neg_mean_absolute_error')
    _, r2, __ = cv_avg_std(reg, X_T, y, 'r2')
  
    print('AVG_MAE', avg, 'R2', r2, 'std_dev', std_dev, "[", n_estimators, ",", min_samples_leaf, ",", min_samples_split, ",", max_depth, ',', max_features, ',', warm_start, ',', bootstrap, '] - ', rand_n)

    return float('inf') if std_dev > 75 else -r2

instru = ng.p.Instrumentation(
    ng.p.Choice([x for x in range(40, 200)]), # n_estimators
    ng.p.Choice([x for x in range(1, 4)]), # min_samples_leaf
    ng.p.Choice([x for x in range(2, 5)]), # min_samples_split
    ng.p.Choice([x for x in range(30, 200)]), # max_depth
    ng.p.Choice([x for x in range(10, 200)]), # max_features
    ng.p.Choice([True, False]), # warm_start
    ng.p.Choice([True, False]) # bootstrap
)

# Descomentar para ejecutar

#optimizer = ng.optimizers.CM(parametrization=instru, budget=1000)
#recommendation = optimizer.minimize(optimize_spectral)
#print(recommendation.value)  # recommended value

<center><h2><b>Reducción dimensional</b></h2></center>

Hemos probado SVD para reducir el número de features, acelerar el tiempo de entreno y reducir el ruido que se haya podido originar<br>
por parte de features que no sean demasiados útiles, ya que nuestro conjunto de training es una matriz esparsa y no se puede usar PCA,<br>
sin embargo, no ha ayudado a mejorar el modelo, incrementando en su vez todas las medidas de error.

In [ ]:
def feature_n_for_explained_variance_ratio(svd, ratio):
    total = 0
    
    for i, n in enumerate(svd.explained_variance_ratio_):
        total += n
        
        if total >= ratio:
            return i+1
    
    return -1


from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=22)
_X_T_train = svd.fit_transform(X_T_train)

for n in (0.5, 0.75, 0.9, 0.99, 0.999, 0.9999):
    print("Numero de features para explicar un " + str(n*100) + "% de la varianza:",
          feature_n_for_explained_variance_ratio(svd, n))

#X_T = svd.transform(X_T)

<center><h2><b>Entrenamiento y calcular Error</b></h2></center>

Para el cálculo de error hemos usado tres medidas. Por una parte tenemos el MAE para calcular la distancia media a la variable objetivo que es el<br>
número de minutos que tenemos que predecir que un paciente va a permanecer en la UCI. Aparte del MAE también hemos considerado importante la desviación<br>
estandar del mismo ya que se trata de un sistema donde se pueden perder vidas si se hacen predicciones poco precisas. Por último hemos usado R2 para<br>
calcular la cantidad de varianza que explica el modelo y evitar overfitting.

In [ ]:
# avg df: 3484.0752 = 2.420139 dias.
def cv_avg_std(reg, X, y, scoring):
    maes = cross_val_score(reg, X, y, cv=2, scoring=scoring)
    avg = stat.mean(maes)
    std_dev = stat.variance(maes)**(1/2)
    
    return maes, avg, std_dev

def make_df(datos_reg):
    error_df = pd.DataFrame()

    error_df['Regresor']               = datos_reg.keys()
    error_df['Average MAE']            = [ abs(dato['avg']) for dato in datos_reg.values() ]
    error_df['Standard Deviation MAE'] = [ dato['std_dev'] for dato in datos_reg.values() ]
    error_df['Average R2']             = [ dato['avg_r2'] for dato in datos_reg.values() ]
    error_df['time']                   = [ dato['time'] for dato in datos_reg.values() ]
    
    return error_df

In [ ]:
# Linear regression 
lreg = LinearRegression()

rfreg = RandomForestRegressor(
        n_estimators = 121,
        min_samples_leaf = 2,
        min_samples_split = 3,
        max_depth = 91,
        max_features = 189,
        warm_start = False,
        bootstrap = False,
        random_state = 14684358)

xboostreg = XGBRegressor()

votingreg = VotingRegressor([('xboost', lreg), ('rfreg', rfreg)], weights=[1, 3])

datos_reg = {}
regressors = [
    ('Lineal', lreg),
    ('Random Forest', rfreg),
    ('XGBRegressor', xboostreg),
    ('VotingRegressor', votingreg),
]

# Medir tiempo y hacer predicciones para cada regresor
for reg_name, reg in regressors:
    start_time = time.time()

    maes, avg, std_dev = cv_avg_std(reg, X_T_train, y_train, 'neg_mean_absolute_error')
    maes, r2, _ = cv_avg_std(reg, X_T_train, y_train, 'r2')
    
    datos_reg[reg_name] = { 'avg': avg, 'std_dev': std_dev, 'time': time.time() - start_time }
    datos_reg[reg_name]['avg_r2'] = r2

df_reg = make_df(datos_reg)
df_reg

In [ ]:
sns.set_palette("Paired")
fig, axs = plt.subplots(ncols=3, nrows=1,figsize=(20, 5))

sns.barplot(data=df_reg, x='Regresor', y='Average MAE', ax=axs[0])
sns.barplot(data=df_reg, x='Regresor', y='Standard Deviation MAE', ax=axs[1])
sns.barplot(data=df_reg, x='Regresor', y='Average R2', ax=axs[2])

Por último hacemos un plot con el error de cada modelo para comparar cuanto se distancian de un resultado ideal.

In [ ]:
rfreg.fit(X_T_train, y_train)
pred = rfreg.predict(X_T_test)

In [ ]:
# avg df: 3484.0752 = 2.420139 dias.
mae = mean_absolute_error(y_test, pred)
r2  = r2_score(y_test, pred)

print('MAE test:', mae)
print('R2:', r2, '\n')

err = prediction_error(votingreg,  X_T_train, y_train, X_T_test, y_test)

Basandonos en estos plots podemos ver que todos se acercan bastante a la pendiente de 45º. LinearRegressor y GBRegressor se<br>
alejan un poco más, pero es natural ya que no hemos tenido tiempo de buscar unos hiperparámetros razonables o hacer eliminación<br>
de outliers. En el caso de RandomForest no le afectan tanto estos outliers y se hace posible una explicación más amplia de la varianza.

---
<center><h2><b>Conclusión</b></h2></center>

Tras evaluar todos estos modelos podemos observar que mientras que LinearRegressor tiene un ligeramente mayor R2 y un tiempo de ejecución<br>
envidiable, la diferencia en MAE no compensa usarlo. RandomForestRegressor se muestra como un contendiente fuerte, pero hemos podido mejorarlo<br>
haciendo un ensemble de Voting con el anterior. XBoost probablemente mejoraría dado el tiempo suficiente para hacer una búsqueda de hiperparámetros<br>
exhaustiva, pero no hemos tenido tiempo. A todo esto pensamos que el VotingRegressor muestra unos resultados aceptables.<br>

En un nivel más abstracto podemos ver que tiene una media de error de ~262 minutos ~ 4 horas y una desviación estandar de alrededor de media hora. Creemos<br>
que es importante contar con una desviación estandar pequeña para poder predecir la estancia de los pacientes con cierta fiabilidad y teniendo en cuenta<br>
que el tiempo medio que un paciente está en la UCI en E.E.U.U es de alrededor de 10 días y que la media del dataset es de 2.4 días, creemos que es un<br>
resultado interesante con el cual se podrían hacer, hipotéticamente, predicciones útiles.<br>

También es merecedor de mención que este resultado no supera a APACHE. Un modelo únicamente entrenado con la variable unbridgedunitlos que tiene un índice<br>
de correlación de más del 98% con la variable que estamos buscando tiene un R2 de un 98.5% y un MAE y desviación estandar menores. No hemos querido mejorar<br>
el modelo hasta esos niveles tanto por falta de tiempo como por miedo de overfitting, aunque Random Forest es notorio por su tendencia a evitar overfitting.<br>

En general y para terminar podemos decir que ha sido un reto interesante donde nuestros principales problemas han sido la interpretación de 30 tablas de datos<br>
(muchas para la experiencia que habíamos tenido hasta ahora) y la interpretación de dichos datos, sobretodo los datos vacíos (donde algunos han acabado teniendo<br>
sentido) como el significado de dichos datos en un contexto médico.